# Lab 05: Logistic Regression

## Reminder: Logistic Regression
- binary classification 문제

### Hypothesis
$$ H(X) = \frac{1}{1+e^{-W^T X}} $$
<br/>
- m개의 샘플인 $X$가 들어왔을 때 $H(x)$가 0 또는 1이 되도록 하고싶음.
<br/> <br/>
$$\simeq P(X=1)$$ <br/>
$$\frac{1}{1+e^{-X W}} $$ <br/>
- $|XW| = (m, d)$ x $(d, 1) = (m, 1)$
- $d$차원의 $1d$ 벡터가 주어졌을 때 0과 1 중 어느 쪽에 더 가까운지 구하고싶음.

### Cost
$$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $$
<br/>
 - If $y \simeq H(x)$, cost is near 0.
 - If $y \neq H(x)$, cost is high.

### Weight Update via Gradient Descent
$$ W := W - \alpha \frac{\partial}{\partial W} cost(W)  $$
<br/>
 - $\alpha$: Learning rate
 - weight를 최소화하는 방향으로 업데이트가 될 것 이다.

## Imports

In [63]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [64]:
# For reproducibility
torch.manual_seed(1) # 결과를 늘 똑같이 재연하기 위해

## Training Data

In [65]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

- |x_data| = (6, 2)
    - m=6 / d=2
- |y_data| = (6,)

In [66]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

As always, we need these data to be in `torch.Tensor` format, so we convert them.

In [67]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


## Computing the Hypothesis

$$ H(X) = \frac{1}{1+e^{-W^T X}} => \frac{1}{1+e^{-X W}} => \frac{1}{1+e^{-X W + b}}$$

#### PyTorch has a `torch.exp()` function.

In [68]:
# e^1을 출력해보자.
print('e^1 equlas: ', torch.exp(torch.FloatTensor([1])))

e^1 equlas:  tensor([2.7183])


We can use it to compute the hypothesis function conveniently.

In [69]:
W = torch.zeros((2, 1), requires_grad=True) # d=2
b = torch.zeros(1, requires_grad=True)

In [70]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b))) # torch.matmul(X, W)

In [71]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


$W$를 모두 0으로 setting 했기 때문에 $H(x)$의 값은 늘 0.5가 나오겠지 !!

#### Or, `torch.sigmoid()` function

In [72]:
print('1/(1+e^{-1}) equlas: ', torch.sigmoid(torch.FloatTensor([1])))

1/(1+e^{-1}) equlas:  tensor([0.7311])


In [73]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [74]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


## Computing the Cost Function (Low-level)

$$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $$

#### measure the difference between `hypothesis` and `y_train`.

In [75]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [76]:
# For one element,
-(y_train[0] * torch.log(hypothesis[0]) + 
 (1-y_train[0]) * torch.log(1-hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward0>)

In [77]:
# Input the entire vector.
losses = -(y_train * torch.log(hypothesis) + 
          (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


#### Use `.mean()`

In [78]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


## Computing the Cost Function with `F.binary_cross_entropy`

위 과정을 한 번에 할 수 있는 `F.binary_cross_entropy` 함수

In [79]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

## Training with Low-level Binary Cross Entropy loss

In [80]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [81]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1) # W, b를 훈련할 거야

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    
    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = -(y_train * torch.log(hypothesis) + 
            (1 - y_train) * torch.log(1 - hypothesis)).mean()
    
    # Cost로 H(x) 개선
    optimizer.zero_grad() # 초기화 안 하면 gradient 구할 때마다 더해짐
    cost.backward()
    optimizer.step() # cost를 minimize하는 방향으로
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


## Training with `F.binary_cross_entropy`

In [82]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    
    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031672
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


## Loading Real Data

In [83]:
import numpy as np

In [84]:
xy = np.loadtxt('data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [85]:
print(x_train[0:5])
print(y_train[0:5])

tensor([[-0.2941,  0.4874,  0.1803, -0.2929,  0.0000,  0.0015, -0.5312, -0.0333],
        [-0.8824, -0.1457,  0.0820, -0.4141,  0.0000, -0.2072, -0.7669, -0.6667],
        [-0.0588,  0.8392,  0.0492,  0.0000,  0.0000, -0.3055, -0.4927, -0.6333],
        [-0.8824, -0.1055,  0.0820, -0.5354, -0.7778, -0.1624, -0.9240,  0.0000],
        [ 0.0000,  0.3769, -0.3443, -0.2929, -0.6028,  0.2846,  0.8873, -0.6000]])
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


In [86]:
print(x_train.shape)
print(y_train.shape)

torch.Size([759, 8])
torch.Size([759, 1])


## Training with Real Data usign low-level Binary Cross Entropy Loss

In [87]:
# 모델 초기화
W = torch.zeros((8, 1), requires_grad=True) # dim=8
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 100
for epoch in range(nb_epochs+1):
    
    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)).mean()
    
    # Cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 10번마다 로그 출력
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/100 Cost: 0.693147
Epoch   10/100 Cost: 0.572727
Epoch   20/100 Cost: 0.539493
Epoch   30/100 Cost: 0.519708
Epoch   40/100 Cost: 0.507066
Epoch   50/100 Cost: 0.498539
Epoch   60/100 Cost: 0.492549
Epoch   70/100 Cost: 0.488209
Epoch   80/100 Cost: 0.484985
Epoch   90/100 Cost: 0.482543
Epoch  100/100 Cost: 0.480661


## Training with Real Data usign `F.binary_cross_entropy`

In [88]:
# 모델 초기화
W = torch.zeros((8, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/100 Cost: 0.693147
Epoch   10/100 Cost: 0.572727
Epoch   20/100 Cost: 0.539493
Epoch   30/100 Cost: 0.519708
Epoch   40/100 Cost: 0.507066
Epoch   50/100 Cost: 0.498539
Epoch   60/100 Cost: 0.492549
Epoch   70/100 Cost: 0.488209
Epoch   80/100 Cost: 0.484985
Epoch   90/100 Cost: 0.482543
Epoch  100/100 Cost: 0.480661


## Checking the Accuracy our Model
want to check how well our model fits the training data

In [89]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[0.4103],
        [0.9242],
        [0.2300],
        [0.9411],
        [0.1772]], grad_fn=<SliceBackward0>)


We chan change **hypothesis** (R from 0 to 1) to **binary predictions** (either 0 or 1) by comparing them to 0.5.

In [90]:
prediction = hypothesis >= torch.FloatTensor([0.5]) # 크면 true
prediction = prediction.float()
print(prediction[:5])

tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


#### compare with `y_train`.

In [91]:
print(prediction[:5])
print(y_train[:5])

tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


In [92]:
correct_prediction = prediction.float() == y_train
correct_prediction = correct_prediction.float()
print(correct_prediction[:5])

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]])


Calcuate accuracy by the number of correct predictions & dividing by total predictions.

In [93]:
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuarcy of {:2.2f}% for the training set.'.format(accuracy * 100))

The model has an accuarcy of 76.68% for the training set.


## Optional: High-level Implementation with nn.Module
- 실전에서는 이렇게 사용

In [94]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid() # 추가

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [95]:
model = BinaryClassifier()

In [96]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    hypothesis = model(x_train)
    
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(X) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 10번마다 로그 출력
    if epoch % 10 == 0:
        preidction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/100 Cost: 0.704829 Accuracy 76.68%
Epoch   10/100 Cost: 0.572392 Accuracy 76.68%
Epoch   20/100 Cost: 0.539563 Accuracy 76.68%
Epoch   30/100 Cost: 0.520041 Accuracy 76.68%
Epoch   40/100 Cost: 0.507561 Accuracy 76.68%
Epoch   50/100 Cost: 0.499125 Accuracy 76.68%
Epoch   60/100 Cost: 0.493177 Accuracy 76.68%
Epoch   70/100 Cost: 0.488846 Accuracy 76.68%
Epoch   80/100 Cost: 0.485612 Accuracy 76.68%
Epoch   90/100 Cost: 0.483146 Accuracy 76.68%
Epoch  100/100 Cost: 0.481234 Accuracy 76.68%


- `model.parameters()`: model에 있는 W, b 파라미터들이 iterator 형태로 들어옴.